# MARS

In [0]:
!pip install scikit-learn
from sklearn import pyearth

ImportError: ignored

In [0]:
import numpy
from pyearth import Earth
from matplotlib import pyplot

#Create some fake data
numpy.random.seed(0)
m = 1000
n = 10
X = 80*numpy.random.uniform(size=(m,n)) - 40
y = numpy.abs(X[:,6] - 4.0) + 1*numpy.random.normal(size=m)

#Fit an Earth model
model = Earth()
model.fit(X,y)

#Print the model
print(model.trace())
print(model.summary())

#Plot the model
y_hat = model.predict(X)
pyplot.figure()
pyplot.plot(X[:,6],y,'r.')
pyplot.plot(X[:,6],y_hat,'b.')
pyplot.xlabel('x_6')
pyplot.ylabel('y')
pyplot.title('Simple Earth Example')
pyplot.show()

SyntaxError: ignored

## Preparando ambiente e banco de dados/treino e teste

In [0]:
import pandas as pd


import numpy as np
from scipy.stats import skew
from scipy.stats.stats import pearsonr


train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")


all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
test.loc[:,'MSSubClass':'SaleCondition']))

##  Começamos com algum processamento de dados básico. 
- Primeiro, vou transformar os recursos numéricos distorcidos usando \ log (feature + 1), isso tornará os recursos mais normais
- Criar variáveis Dummy para os recursos categóricos
- Substituir os valores omissos numéricos (NaN’s) pela média de suas respectivas colunas

In [0]:
#log transform the target:
train["SalePrice"] = np.log1p(train["SalePrice"])


#log transform skewed numeric features:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index


skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats &gt; 0.75]
skewed_feats = skewed_feats.index


all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

## Verificar a performance do algorítmo

In [0]:
from sklearn.model_selection import cross_val_score
from pyearth import Earth

def rmse_cv(model):
rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
return(rmse)

mars = Earth(max_degree=1, penalty=1.0, endspan=5)

cv_mars = rmse_cv(mars).mean()

print(cv_mars)

## Indo além

In [0]:
from sklearn.ensemble import AdaBoostRegressor


boosted_model = AdaBoostRegressor(base_estimator=mars, n_estimators=25, learning_rate=0.1, loss="ex